In [1]:
#!pip install iobes
#!pip install seqeval
#!pip install sklearn_crfsuite 
#!conda install spacy
#!pip uninstall numpy
#!pip install numpy

import spacy
from spacy.tokens import Doc, SpanGroup
from spacy.matcher import Matcher
from zipfile import ZipFile
from pathlib import Path
#from tqdm import autonotebook as tqdm
from tqdm import tqdm
from spacy.training import biluo_tags_to_spans
import iobes
import re
from itertools import combinations

import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from seqeval import scheme

import sklearn_crfsuite
from collections import Counter


In [2]:
#!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [3]:
data_dir = Path("./data/teaching-dataset")
with (data_dir / "relation_classification_text_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        sentences = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]
with (data_dir / "relation_classification_references_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("references.txt") as f:
        labels = []
        for line in f.read().decode("utf-8").split("\n"):
            relations = []
            for relation in re.finditer(r"\(\((\d+),(\d+)\),\((\d+),(\d+)\)\)", line):
                relation = (
                    (int(relation.group(1)), int(relation.group(2))),
                    (int(relation.group(3)), int(relation.group(4))),
                )
                relations.append(relation)
            labels.append(relations)

In [4]:
def parse_sentence(sentence):
    words = []
    tags = []
    for item in sentence:
        word, tag = item.split(" ")
        words.append(word)
        tags.append(tag)
    doc = Doc(nlp.vocab, words=words)
    doc = nlp(doc)
    tags = iobes.bio_to_bilou(tags)
    doc.ents = biluo_tags_to_spans(doc, tags)
    return doc

In [5]:
wordlist = []
biggerlist = []
for word in sentences[0]:
    #print(word)
    wordlist.append(str(word.split()[0]))
print(wordlist)
biggerlist.append(wordlist)
biggerlist.append('test')
print(biggerlist)

['According', 'two', 'different', 'studies', 'it', 'seems', 'plausible', 'that', 'the', 'Pohang', 'earthquake', 'was', 'induced', 'by', 'EGS', 'operations', '.']
[['According', 'two', 'different', 'studies', 'it', 'seems', 'plausible', 'that', 'the', 'Pohang', 'earthquake', 'was', 'induced', 'by', 'EGS', 'operations', '.'], 'test']


In [6]:
#wordlist = []
#taglist = []
#biggertaglist = []
#biggerlist = []
#for i in range(len(sentences)):
#    for word in sentences[i]:
#        wordlist.append(str(word.split()[0]))
#        taglist.append(str(word.split()[1]))

        
#    biggerlist.append(wordlist)
#    biggertaglist.append(taglist)
    
#    wordlist = []
#    taglist = []
#print(biggerlist)        

In [7]:
#for word in sentences[0]:
#    print(word.split())
#    print(word.split()[0])
#    print(word.split()[1])

In [8]:
#labels[0]
# get list of all sentences with multiple relationships
helperlist = []
for i in range(len(sentences)):
    if len(labels[i]) > 1:
        helperlist.append(int(i))
print(helperlist)

[2, 3, 8, 9, 10, 12, 13, 18, 19, 20, 21, 22, 25, 27, 28, 30, 33, 34, 35, 36, 40, 43, 45, 47, 50, 51, 61, 64, 65, 67, 70, 72, 74, 77, 78, 79, 81, 82, 85, 86, 90, 91, 93, 95, 97, 99, 100, 102, 103, 104, 105, 106, 107, 108, 111, 112, 113, 114, 115, 118, 121, 123, 128, 130, 136, 138, 140, 145, 146, 149, 151, 155, 157, 159, 160, 170, 173, 174, 177, 179, 180, 182, 183, 185, 193, 194, 195, 197, 200, 202, 205, 209, 210, 211, 212, 220, 227, 228, 229, 230, 232, 233, 236, 238, 239, 240, 241, 246, 249, 252, 262, 264, 266, 268, 271, 275, 277, 279, 280, 284, 288, 294, 296, 301, 302, 303, 305, 307, 309, 312, 314, 316, 317, 320, 322, 323, 324, 325, 335, 338, 340, 343, 344, 346, 347, 349, 357, 359, 362, 369, 372, 373, 374, 382, 385, 388, 389, 390, 391, 392, 393, 394, 396, 397, 400, 401, 403, 407, 409, 410, 411, 413, 415, 422, 427, 433, 434, 437, 441, 442, 446, 448, 449, 450, 451, 452, 453, 454, 455, 456, 458, 460, 464, 466, 467]


In [9]:
# for each sentence, get list of event spans 
helperarray = np.empty([10000,2])
counter2 = 0
for i in range(len(sentences)):
    #print(labels[i])
    for entries in labels[i]:
        #print(entries)
        for numbers in entries:
            #print(numbers)
            for singlets in numbers:
                #print(i, singlets)
                helperarray[counter2] = [i, singlets]
                counter2 += 1
print(helperarray)

[[0.00000000e+000 1.40000000e+001]
 [0.00000000e+000 1.60000000e+001]
 [0.00000000e+000 9.00000000e+000]
 ...
 [5.06096000e-259 1.43711747e+246]
 [2.88970519e+159 4.17699191e+228]
 [1.46292066e+195 3.17091486e+180]]


In [10]:
# get list of event positions for each sentence
linelist = []
for lines in helperarray:
    if lines[0] == 2.0:
        #print(lines)
        linelist.append(lines[1])
for word in sentences[2]:
    print(word)
print(linelist)

Pneumatic B-EVENT
drilling I-EVENT
in I-EVENT
mines I-EVENT
and O
less O
commonly O
, O
mining B-EVENT
using I-EVENT
explosives I-EVENT
, O
would O
raise O
fine B-EVENT
- I-EVENT
ultra I-EVENT
fine I-EVENT
crystalline I-EVENT
silica I-EVENT
dust I-EVENT
( O
rock O
dust O
) O
. O
[0.0, 4.0, 14.0, 21.0, 8.0, 11.0, 14.0, 21.0]


In [11]:
# pop 4 instances from linelist
splitSentenceList = []
splitSentenceList.append(linelist.pop(0))
splitSentenceList.append(linelist.pop(0))
splitSentenceList.append(linelist.pop(0))
splitSentenceList.append(linelist.pop(0))
print(splitSentenceList)
print(linelist)

[0.0, 4.0, 14.0, 21.0]
[8.0, 11.0, 14.0, 21.0]


In [12]:
print(sentences[2])
#print(len(sentences[2]))
original_sentence = sentences[2]
modified_sentences = [None] * len(sentences[2])
for i in range(len(sentences[2])):
    if i not in range(int(splitSentenceList[0]), int(splitSentenceList[1])) and i not in range(int(splitSentenceList[2]), int(splitSentenceList[3])):
        w, t = sentences[2][i].split(' ')
        t = 'O'
        modified_sentences[i] = ' '.join((w,t))
    else:
        modified_sentences[i] = sentences[2][i]

print(sentences[2])
#modified_sentences.append(sentences[2])
print(modified_sentences)
sentences[2] = original_sentence

['Pneumatic B-EVENT', 'drilling I-EVENT', 'in I-EVENT', 'mines I-EVENT', 'and O', 'less O', 'commonly O', ', O', 'mining B-EVENT', 'using I-EVENT', 'explosives I-EVENT', ', O', 'would O', 'raise O', 'fine B-EVENT', '- I-EVENT', 'ultra I-EVENT', 'fine I-EVENT', 'crystalline I-EVENT', 'silica I-EVENT', 'dust I-EVENT', '( O', 'rock O', 'dust O', ') O', '. O']
['Pneumatic B-EVENT', 'drilling I-EVENT', 'in I-EVENT', 'mines I-EVENT', 'and O', 'less O', 'commonly O', ', O', 'mining B-EVENT', 'using I-EVENT', 'explosives I-EVENT', ', O', 'would O', 'raise O', 'fine B-EVENT', '- I-EVENT', 'ultra I-EVENT', 'fine I-EVENT', 'crystalline I-EVENT', 'silica I-EVENT', 'dust I-EVENT', '( O', 'rock O', 'dust O', ') O', '. O']
['Pneumatic B-EVENT', 'drilling I-EVENT', 'in I-EVENT', 'mines I-EVENT', 'and O', 'less O', 'commonly O', ', O', 'mining O', 'using O', 'explosives O', ', O', 'would O', 'raise O', 'fine B-EVENT', '- I-EVENT', 'ultra I-EVENT', 'fine I-EVENT', 'crystalline I-EVENT', 'silica I-EVENT'

In [17]:
emptydict = {'a': 'test'}
emtpydict['0'] = original_sentence
print(emptydict)

NameError: name 'emtpydict' is not defined

In [ ]:
print(original_sentence)
print(modified_sentences[0])

In [ ]:
#print(len(labels[467]))
#print(len(sentences[0]))
##helper = np.zeros((len(sentences[0]), len(sentences[0])))
#helper = np.zeros(len(sentences[0]))
#print(helper)

#for entry in labels[0][0]:
#    print(entry)
#    for numbers in entry:
#        print(numbers)
#        helper[numbers-1] = 1
#print(helper)

In [ ]:
#jcount = 0
#for j in helper:
#    jcount += 1
#    if j > 0:
#        print(jcount)

In [ ]:
#print(labels[1][0][0][0])
#print(labels[1][0][1][0])

In [ ]:
#print(labels[2])
#doc = parse_sentence(sentences[2])
#doc.ents

In [ ]:
#new df with 1 line per sentence 
sentence_number=[]
wordlist = []
taglist = []
POSlist = []
DEPlist = []
customclass = []

biggerlist = []
biggertaglist = []
biggerPOSlist = []
biggerDEPlist = []
for i in range(len(sentences)):
    if labels[i][0][0][0] < labels[0][0][1][0]:
        customclass.append('0')
    elif labels[i][0][0][0] >= labels[0][0][1][0]:
        customclass.append('1')
for i in range(len(sentences)):
    sentence_number.append(str(i))
    
    for word in sentences[i]:
        wordlist.append(str(word.split()[0]))
        taglist.append(str(word.split()[1]))

    doc = parse_sentence(sentences[i])
    for token in doc:
        POSlist.append(token.pos_)
        DEPlist.append(token.dep_)
            

    biggerlist.append(wordlist)
    biggertaglist.append(taglist)
    biggerPOSlist.append(POSlist)
    biggerDEPlist.append(DEPlist)
    
    wordlist = []
    taglist = []
    POSlist = []
    DEPlist = []

train_data = {'Sentence #': sentence_number, 'Wordlist': biggerlist, 'Taglist': biggertaglist, 'POS': biggerPOSlist, 'DEP': biggerDEPlist, 'customclass': customclass,}
#train_data = {'Sentence #': sentence_number, 'Wordlist': str(biggerlist), 'Taglist': str(biggertaglist), 'POS': str(biggerPOSlist), 'DEP': str(biggerDEPlist), 'customclass': customclass,}
df = pd.DataFrame(data=train_data)
print(df)

In [ ]:
df.isnull().sum()

df = df.fillna(method='ffill')
#df['Sentence #'].nunique(), df.Wordlist.nunique(), df.Taglist.nunique(), df.customclass.nunique()
#print(df['Sentence #'].nunique())
#print(df.Wordlist.nunique())
#print(df.Taglist.nunique())
#print(df.CustomClass.nunique())

#df.groupby('Tag').size().reset_index(name='counts')
#print(df.groupby('Tag').size().reset_index(name='counts'))

df.groupby('customclass').size().reset_index(name='counts')
print(df.groupby('customclass').size().reset_index(name='counts'))

#X = df.drop('Tag', axis=1)
X = df.drop('customclass', axis=1)
#print(X)

v = DictVectorizer(sparse=False)

X = v.fit_transform(X.to_dict('records'))

for i in range(10):
    print(X[i])
    
#y = df.Tag.values
y = df.customclass.values
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
X_train.shape, y_train.shape

In [ ]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)
#per.fit(X_train, y_train, classes, coef_init = )

In [ ]:
#per.partial_fit(X_train, y_train, classes)

In [ ]:
new_classes = classes.copy()
print(new_classes)

In [ ]:
print(classification_report(y_pred=per.predict(X_test), y_true=y_test))

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
print(per.predict(X_test))
print(y_test)

In [ ]:
def extract_active_passive(doc):
    # https://stackoverflow.com/questions/74528441/detect-passive-or-active-sentence-from-text
    passive_rules = [
        [
            {"DEP": "nsubjpass"},
            {"DEP": "aux", "OP": "*"},
            {"DEP": "auxpass"},
            {"TAG": "VBN"},
        ],
        [
            {"DEP": "nsubjpass"},
            {"DEP": "aux", "OP": "*"},
            {"DEP": "auxpass"},
            {"TAG": "VBZ"},
        ],
        [
            {"DEP": "nsubjpass"},
            {"DEP": "aux", "OP": "*"},
            {"DEP": "auxpass"},
            {"TAG": "RB"},
            {"TAG": "VBN"},
        ],
    ]
    # Create pattern to match active voice use
    active_rules = [
        [{"DEP": "nsubj"}, {"TAG": "VBD", "DEP": "ROOT"}],
        [{"DEP": "nsubj"}, {"TAG": "VBP"}, {"TAG": "VBG", "OP": "!"}],
        [{"DEP": "nsubj"}, {"DEP": "aux", "OP": "*"}, {"TAG": "VB"}],
        [{"DEP": "nsubj"}, {"DEP": "aux", "OP": "*"}, {"TAG": "VBG"}],
        [{"DEP": "nsubj"}, {"TAG": "RB", "OP": "*"}, {"TAG": "VBG"}],
        [{"DEP": "nsubj"}, {"TAG": "RB", "OP": "*"}, {"TAG": "VBZ"}],
        [{"DEP": "nsubj"}, {"TAG": "RB", "OP": "+"}, {"TAG": "VBD"}],
    ]
    matcher = Matcher(nlp.vocab)
    matcher.add("Passive", passive_rules)
    matcher.add("Active", active_rules)
    matches = matcher(doc)
    matches = [
        (nlp.vocab.strings[match_id], doc[start:end])
        for match_id, start, end in matches
    ]
    return matches


def predict(sentence):
    doc = parse_sentence(sentence)
    matches = extract_active_passive(doc)
    predictions = []
    for ent_1, ent_2 in combinations(doc.ents, 2):
        for match_type, match_span in matches:
            if SpanGroup(doc, spans=[ent_1, ent_2, match_span]).has_overlap:
                match_active = match_type == "Active"
                if match_active:
                    predictions.append(
                        ((ent_1.start, ent_1.end), (ent_2.start, ent_2.end))
                    )
                    break
                else:
                    predictions.append(
                        ((ent_2.start, ent_2.end), (ent_1.start, ent_1.end))
                    )
                    break
    return predictions

In [ ]:
doc = parse_sentence(sentences[0])
matches = extract_active_passive(doc)
print(doc)
for match_type, match_span in matches:
    print("\t{}: {}".format(match_type, match_span.text))

In [ ]:
idx = 5
doc = parse_sentence(sentences[idx])
pred = predict(sentences[idx])
print(doc)
print("Ground truth:")
for cause, effect in labels[idx]:
    print("\t{} -> {}".format(doc[cause[0]:cause[1]], doc[effect[0]:effect[1]]))
print("Predictions:")
for cause, effect in pred:
    print("\t{} -> {}".format(doc[cause[0]:cause[1]], doc[effect[0]:effect[1]]))


In [ ]:
predictions = []
#for sentence in tqdm.tqdm(sentences):
for sentence in tqdm(sentences):
    predictions.append(predict(sentence))

In [ ]:
def evaluate(predictions, references, micro_avg=True):
    tp = []
    fp = []
    fn = []
    for prediction, reference in zip(predictions, references):
        tp.append(len(set(prediction) & set(reference)))
        fp.append(len(set(prediction) - set(reference)))
        fn.append(len(set(reference) - set(prediction)))
    if micro_avg:
        tp = [sum(tp)]
        fp = [sum(fp)]
        fn = [sum(fn)]
    precision = [0 if tp[i] == 0 else tp[i] / (tp[i] + fp[i]) for i in range(len(tp))]
    recall = [0 if tp[i] == 0 else tp[i] / (tp[i] + fn[i]) for i in range(len(tp))]
    f1 = [
        0
        if precision[i] * recall[i] == 0
        else 2 * precision[i] * recall[i] / (precision[i] + recall[i])
        for i in range(len(tp))
    ]
    precision = sum(precision) / len(precision)
    recall = sum(recall) / len(recall)
    f1 = sum(f1) / len(f1)
    return precision, recall, f1


micro_precision, micro_recall, micro_f1 = evaluate(predictions, labels, True)
macro_precision, macro_recall, macro_f1 = evaluate(predictions, labels, False)

print("Micro Precision: {:.2f}".format(micro_precision))
print("Micro Recall: {:.2f}".format(micro_recall))
print("Micro F1: {:.2f}".format(micro_f1))
print("Macro Precision: {:.2f}".format(macro_precision))
print("Macro Recall: {:.2f}".format(macro_recall))
print("Macro F1: {:.2f}".format(macro_f1))
